<a href="https://colab.research.google.com/github/liudiepie/Introduction_to_Machine_Learning/blob/main/Assignment_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing(NLP)

### Loading AG News with Torchtext

In [1]:
!pip install torchtext==0.8.1

     |████████████████████████████████| 7.0 MB 13.4 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.7.1 which is incompatible.


In [2]:
import torchtext
agnews_train, agnews_test = torchtext.datasets.text_classification.DATASETS["AG_NEWS"](root="./datasets")

ag_news_csv.tar.gz: 11.8MB [00:00, 52.7MB/s]
120000lines [00:05, 23455.90lines/s]
120000lines [00:11, 10593.13lines/s]
7600lines [00:00, 11025.09lines/s]


In [ ]:
#this code for loading datasets from own computer
"""
import torchtext

ngrams = 1
train_csv_path = './datasets/ag_news_csv/train.csv'
test_csv_path = './datasets/ag_news_csv/test.csv'
vocab = torchtext.vocab.build_vocab_from_iterator(
    torchtext.legacy.datasets.text_classification._csv_iterator(train_csv_path, ngrams))
train_data, train_labels = torchtext.legacy.datasets.text_classification._create_data_from_iterator(
        vocab, torchtext.legacy.datasets.text_classification._csv_iterator(train_csv_path, ngrams, yield_cls=True), False)
test_data, test_labels = torchtext.legacy.datasets.text_classification._create_data_from_iterator(
        vocab, torchtext.legacy.datasets.text_classification._csv_iterator(test_csv_path, ngrams, yield_cls=True), False)
if len(train_labels ^ test_labels) > 0:
    raise ValueError("Training and test labels don't match")
agnews_train = torchtext.legacy.datasets.TextClassificationDataset(vocab, train_data, train_labels)
agnews_test = torchtext.legacy.datasets.TextClassificationDataset(vocab, test_data, test_labels)
"""

'\nimport torchtext\n\nngrams = 1\ntrain_csv_path = \'./datasets/ag_news_csv/train.csv\'\ntest_csv_path = \'./datasets/ag_news_csv/test.csv\'\nvocab = torchtext.vocab.build_vocab_from_iterator(\n    torchtext.legacy.datasets.text_classification._csv_iterator(train_csv_path, ngrams))\ntrain_data, train_labels = torchtext.legacy.datasets.text_classification._create_data_from_iterator(\n        vocab, torchtext.legacy.datasets.text_classification._csv_iterator(train_csv_path, ngrams, yield_cls=True), False)\ntest_data, test_labels = torchtext.legacy.datasets.text_classification._create_data_from_iterator(\n        vocab, torchtext.legacy.datasets.text_classification._csv_iterator(test_csv_path, ngrams, yield_cls=True), False)\nif len(train_labels ^ test_labels) > 0:\n    raise ValueError("Training and test labels don\'t match")\nagnews_train = torchtext.legacy.datasets.TextClassificationDataset(vocab, train_data, train_labels)\nagnews_test = torchtext.legacy.datasets.TextClassificationDat

In [3]:
print(agnews_train[0])

(2, tensor([  432,   426,     2,  1606, 14839,   114,    67,     3,   849,    14,
           28,    15,    28,    16, 50726,     4,   432,   375,    17,    10,
        67508,     7, 52259,     4,    43,  4010,   784,   326,     2]))


In [4]:
print("Length of the first text example: {}".format(len(agnews_train[0][1])))
print("Length of the second text example: {}".format(len(agnews_train[1][1])))

Length of the first text example: 29
Length of the second text example: 42


In [5]:
from torch.nn.utils.rnn import pad_sequence

padded_exs = pad_sequence([agnews_train[0][1], agnews_train[1][1]])
print("First sequence padded: {}".format(padded_exs[:,0]))
print("First sequence length: {}".format(len(padded_exs[:,0])))
print("Second sequence padded: {}".format(padded_exs[:,1]))
print("Second sequence length: {}".format(len(padded_exs[:,1])))

First sequence padded: tensor([  432,   426,     2,  1606, 14839,   114,    67,     3,   849,    14,
           28,    15,    28,    16, 50726,     4,   432,   375,    17,    10,
        67508,     7, 52259,     4,    43,  4010,   784,   326,     2,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0])
First sequence length: 42
Second sequence padded: tensor([15875,  1073,   855,  1311,  4251,    14,    28,    15,    28,    16,
          930,   798,   321, 15875,    99,     4, 27658,    29,     6,  4460,
           12,   565, 52791,     9, 80618,  2126,     8,     3,   526,   242,
            4,    29,  3891, 82815,  6575,    11,   207,   360,     7,     3,
          127,     2])
Second sequence length: 42


In [6]:
import numpy as np
import torch

def collator(batch):
    labels = torch.tensor([example[0] for example in batch])
    sentences = [example[1] for example in batch]
    data = pad_sequence(sentences)
    
    return [data, labels]

In [7]:
BATCH_SIZE = 128

train_loader = torch.utils.data.DataLoader(agnews_train, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collator)
test_loader = torch.utils.data.DataLoader(agnews_test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collator)

### Simple Word Embedding Model

In [8]:
VOCAB_SIZE = len(agnews_train.get_vocab())
EMBED_DIM = 100
HIDDEN_DIM = 64
NUM_OUTPUTS = len(agnews_train.get_labels())
NUM_EPOCHS = 3

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class SWEM(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_dim, num_outputs):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        self.fc1 = nn.Linear(embedding_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_outputs)

    def forward(self, x):
        embed = self.embedding(x)
        embed_mean = torch.mean(embed, dim=0)
        
        h = self.fc1(embed_mean)
        h = F.relu(h)
        h = self.fc2(h)
        return h

In [11]:
## Training
# Instantiate model
model = SWEM(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, NUM_OUTPUTS)

# Binary Cross Entropy Loss and Adam Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Iterate through train set minibatchs 
for epoch in range(250):
    correct = 0
    num_examples = 0
    for inputs, labels in train_loader:
        # Zero out the gradients
        optimizer.zero_grad()
        
        # Forward pass
        y = model(inputs)
        loss = criterion(y, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        predictions = y.argmax(dim=1)
        correct += (predictions == labels).sum().item()
        num_examples += labels.size(0)
    
    # Print training progress
    if epoch % 10 == 0:
        acc = correct/num_examples
        print("Epoch: {0} \t Train Loss: {1} \t Train Acc: {2}".format(epoch, loss, acc))

## Testing
correct = 0
num_test = 0

with torch.no_grad():
    # Iterate through test set minibatchs 
    for inputs, labels in test_loader:
        # Forward pass
        y = model(inputs)
        
        predictions = y.argmax(dim=1)
        correct += (predictions == labels).sum().item()
        num_examples += labels.size(0)
    
print('Test accuracy: {}'.format(correct/num_test))

Epoch: 0 	 Train Loss: 0.4672899544239044 	 Train Acc: 0.7307166666666667
Epoch: 10 	 Train Loss: 0.025780154392123222 	 Train Acc: 0.9743666666666667
Epoch: 20 	 Train Loss: 0.0004224234726279974 	 Train Acc: 0.99125
Epoch: 30 	 Train Loss: 0.031344421207904816 	 Train Acc: 0.9951416666666667
Epoch: 40 	 Train Loss: 6.0853806644445285e-05 	 Train Acc: 0.9965
Epoch: 50 	 Train Loss: 0.0005616910057142377 	 Train Acc: 0.9977083333333333


KeyboardInterrupt: ignored